# LangChain
LangChain is a framework for developing applications powered by language models.

## Schema
Most interfaces in LangChain are text based because most language models support only "text in, text out" (if you are interested, we can have an hour long discussion where I can explain you everything about transformers architecture).

### How LLMs work (practical approach, not theory)
LLMs have 3 parts in prompt usually.
1. System Chain Message
    - A chat message representing information that should be instructions to the AI system.
2. Human Chat Message
    - A chat message representing information coming from a human interacting with the AI system.
3. AI Chat Message
    - A chat message representing information coming from the AI system.

**__PRO_TIP__**: While making product, try to keep Human chat as low as possible because normal users are very bad at writing prompt and model give pretty bad results with bad prompts which will results into you replying to useless complaint emails.

Other than that, one important thing I need to explain.
>    - Our daily usage is when we use model with zero-shot learning but model give outstanding result when it is fine-tuned with some examples, usually known as few-shot learning. We use method of `Examples` for this where we provide model with input/output pairs so model can better understand what we are expecting. 

While talking about terminology, we use word `Document` for unstructured data. Documents have compeletely random formats so normal programming approach cannot get desired data from doc. That is why we use LLMs because they can really understand data and give response accordingly. Document has two parts, `page_content` and `metadata`.

## Models
LangChain deals with 3 types of models.
1. Language Models
2. Chat Models
3. Text Embedding Models

LLMs: <br>
These models take a text string as input, and return a text string as output.<br><br>
Chat Models:<br>
These models are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.<br><br>
Text Embedding Models<br>
These models take text as input and return a list of floats.<br>

### LLMs
Large Language Models (LLMs) are a core component of LangChain. LangChain is not a provider of LLMs, but rather provides a standard interface through which you can interact with a variety of LLMs. The LLM class is a class designed for interfacing with LLMs. There are lots of LLM providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.
<br>
Here is code example:

```
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
llm("Tell me a joke")

Output:
>>> '\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'
```
Let's see how to pass multiple inputs:

```
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"]*15)
len(llm_result.generations)

Output:
>>> 30
```
As llm_result.generations return list, it can be accessed like standard list.

If LLM provider provide usage data, LangChain will return that too but it will not work for all LLMs so consider it not standardized. As our usage is focused on OpenAI, it will work.

```
llm_result.llm_output
```
Results will look like:
```
{'token_usage': {'completion_tokens': 3903,
  'total_tokens': 4023,
  'prompt_tokens': 120}}
```

Further, you can also count number of tokens in query:

```
llm.get_num_tokens("what a joke")
>>> 3
```

### How (and why) to use the fake LLM
We expose a fake LLM class that can be used for testing. This allows you to mock out calls to the LLM and simulate what would happen if the LLM responded in a certain way.

Here is code example:

```
from langchain.llms.fake import FakeListLLM

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["python_repl"])
responses=[
    "Action: Python REPL\nAction Input: print(2 + 2)",
    "Final Answer: 4"
]
llm = FakeListLLM(responses=responses)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("whats 2 + 2")

Output:
> Entering new AgentExecutor chain...
Action: Python REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.

'4'
```

Not writing about `LLM Wrappers`, `Cache LLM calls`, `Stream LLM and chat response` as my team does not require that ATM, will update if required in future. <br>
Let's talk about token usage.

```
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)
```
Output will be:

```
>>> Tokens Used: 42
...  	Prompt Tokens: 4
... 	Completion Tokens: 38
... Successful Requests: 1
... Total Cost (USD): $0.00084
```

Anything inside the context manager will get tracked. Here’s an example of using it to track multiple calls in sequence.

```
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    result2 = llm("Tell me a joke")
    print(cb.total_tokens)
```
Results:

```
>>> 91
```

Sometimes, our code will be using multiple steps, tracking is simple for that too.
Here is code:

```
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

with get_openai_callback() as cb:
    response = agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
```

Results will go something like below:

```
> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: Sudeikis and Wilde's relationship ended in November 2020. Wilde was publicly served with court documents regarding child custody while she was presenting Don't Worry Darling at CinemaCon 2022. In January 2021, Wilde began dating singer Harry Styles after meeting during the filming of Don't Worry Darling.
Thought: I need to find out Harry Styles' age.
Action: Search
Action Input: "Harry Styles age"
Observation: 29 years
Thought: I need to calculate 29 raised to the 0.23 power.
Action: Calculator
Action Input: 29^0.23
Observation: Answer: 2.169459462491557

Thought: I now know the final answer.
Final Answer: Harry Styles, Olivia Wilde's boyfriend, is 29 years old and his age raised to the 0.23 power is 2.169459462491557.

> Finished chain.
Total Tokens: 1506
Prompt Tokens: 1350
Completion Tokens: 156
Total Cost (USD): $0.03012
```

## Integration guide
Although entire guideline is for number of models, I am writing for the models my team will use, ie. OpenAI, Cohere, Hugging face Hub and Hugging face local pipelines

### Cohere

```
>>> pip install cohere
```
After installation, it is standard procedure.
```
# get a new token: https://dashboard.cohere.ai/

from getpass import getpass

COHERE_API_KEY = getpass()

from langchain.llms import Cohere
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = Cohere(cohere_api_key=COHERE_API_KEY)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)
```

Results will go like:

```
" Let's start with the year that Justin Beiber was born. You know that he was born in 1994. We have to go back one year. 1993.\n\n1993 was the year that the Dallas Cowboys won the Super Bowl. They won over the Buffalo Bills in Super Bowl 26.\n\nNow, let's do it backwards. According to our information, the Green Bay Packers last won the Super Bowl in the 2010-2011 season. Now, we can't go back in time, so let's go from 2011 when the Packers won the Super Bowl, back to 1984. That is the year that the Packers won the Super Bowl over the Raiders.\n\nSo, we have the year that Justin Beiber was born, 1994, and the year that the Packers last won the Super Bowl, 2011, and now we have to go in the middle, 1986. That is the year that the New York Giants won the Super Bowl over the Denver Broncos. The Giants won Super Bowl 21.\n\nThe New York Giants won the Super Bowl in 1986. This means that the Green Bay Packers won the Super Bowl in 2011.\n\nDid you get it right? If you are still a bit confused, just try to go back to the question again and review the answer"
```